In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import folium
import geopandas as gpd
from shapely.geometry import box

In [2]:
import matplotlib.font_manager as fm

# 한국어 출력을 위한 폰트 설정
plt.rc('font', family='NanumGothic') 

# 마이너스 기호가 깨지는 것을 방지
plt.rcParams['axes.unicode_minus'] = False

In [3]:
# 송파소방서 비상소화장치
fire_equip = pd.read_excel("data/(송파소방서)비상소화장치.xlsx")
# 송파소방서 소방용수
fire_water = pd.read_excel("data/(송파소방서)소방용수.xlsx")
# 송파구 소방서 안전센터 좌표
fire_station = pd.read_csv("data/송파구 소방서 안전센터 좌표.csv", encoding='euc-kr')
# 음면동 경계
emd = gpd.read_file("data/동경계/동경계_geo.shp", encoding='utf-8')

C:\Users\1qlqj\Desktop\multicamp_semi\multicamp_semi\venv\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\1qlqj\Desktop\multicamp_semi\multicamp_semi\venv\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [4]:
fire_equip.head()

,순번,일련번호,장치번호,설치지역,설치구분,설치유형구분,사용구분,관할서,안전센터,주소,상세위치,좌표X,좌표Y,경위도좌표X,경위도좌표Y
0,1,110085251,37,소방차진입곤란,소방서,일체형,양호,송파소방서,거여119안전센터,송파구 성내천로63길 5,공중전화 옆 좌측 1m,213919.2080,443452.3882,127.157410,37.490364
1,2,110082433,32,주거지역,소방서,일체형,양호,송파소방서,방이119안전센터,송파구 천호대로152길 7,전면 40m,210800.5733,448720.2339,127.122220,37.537875
2,3,110083135,33,시장지역,소방서,일체형,양호,송파소방서,거여119안전센터,송파구 성내천로30길 19,한사랑교회 후면 5m,213289.9280,444338.5081,127.150310,37.498358
3,4,110082437,30,주거지역,소방서,일체형,양호,송파소방서,거여119안전센터,송파구 성내천로29다길 17,0m,213671.4567,444603.1973,127.154630,37.500738
4,5,110082430,29,주거지역,소방서,일체형,양호,송파소방서,거여119안전센터,송파구 성내천로33다길 14,0m,213527.9315,444611.7062,127.153007,37.500817


In [5]:
fire_station.head()

,센터명,위도,경도
0,종합운동장119안전센터,37.513567,127.083810
1,잠실119안전센터,37.514113,127.099505
2,거여119안전센터,37.491833,127.148908
3,방이119안전센터,37.524005,127.121039
4,가락119안전센터,37.492365,127.112201


In [6]:
emd.head()

,구,동,geometry
0,종로구,청운동,"POLYGON ((126.97556 37.58968, 126.97549 37.589..."
1,종로구,신교동,"POLYGON ((126.97031 37.58418, 126.97033 37.584..."
2,종로구,궁정동,"POLYGON ((126.97400 37.58654, 126.97401 37.586..."
3,종로구,효자동,"POLYGON ((126.97356 37.58323, 126.97355 37.582..."
4,종로구,창성동,"POLYGON ((126.97353 37.58182, 126.97354 37.581..."


## 송파구 비상소화장치 시각화

In [7]:
# 기본 지도 생성
map = folium.Map(location=[37.490364, 127.157410], zoom_start=12)

# 설치지역에 따른 마커 색상 정의
colors = {
    '소방차진입곤란': 'red',
    '주거지역': 'blue',
    '시장지역': 'green',
    '영세민밀집': 'purple',
    '소방차진입불가': 'orange'
}

# 데이터셋에서 각 장비 위치에 대한 마커 추가
for index, row in fire_equip.iterrows():
    icon_color = colors.get(row['설치지역'], 'gray')  # 설치지역에 따른 색상 가져오기, 기본값은 'gray'
    popup_html = f"""
    <h4>소방 장비 정보</h4>
    <ul style="margin: 0; padding: 0;">
        <li>설치지역: {row['설치지역']}</li>
        <li>설치유형구분: {row['설치유형구분']}</li>
        <li>상세위치: {row['상세위치']}</li>
        <li>주소: {row['주소']}</li>
    </ul>
    """
    popup = folium.Popup(popup_html, max_width=250)  # 팝업 창의 너비 조절
    folium.Marker(
        location=[row['경위도좌표Y'], row['경위도좌표X']],
        popup=popup,
        tooltip=row['주소'],
        icon=folium.Icon(color=icon_color)
    ).add_to(map)

# 범례 추가
legend_html = '''
<div style="position: fixed; 
     bottom: 50px; left: 50px; width: 250px; height: 170px; 
     background-color: white; border:2px solid rgba(0,0,0,0.2); 
     z-index:9999; font-size:14px; border-radius: 8px; 
     box-shadow: 3px 3px 5px rgba(0,0,0,0.3); padding: 10px;">
     <h4 style="text-align:center; font-size:16px; font-weight: bold; margin-top: 0;">설치지역별 마커 색상</h4>
     &nbsp; 소방차진입곤란: <i style="background:#D33D2A; border-radius: 50%; width: 15px; height: 15px; display: inline-block;"></i> 빨강<br>
     &nbsp; 소방차진입불가: <i style="background:#F0932F; border-radius: 50%; width: 15px; height: 15px; display: inline-block;"></i> 주황<br>
     &nbsp; 시장지역: <i style="background:#73A626; border-radius: 50%; width: 15px; height: 15px; display: inline-block;"></i> 초록<br>
     &nbsp; 주거지역: <i style="background:#3BACD9; border-radius: 50%; width: 15px; height: 15px; display: inline-block;"></i> 파랑<br>
     &nbsp; 영세민밀집: <i style="background:#BF4EAC; border-radius: 50%; width: 15px; height: 15px; display: inline-block;"></i> 보라<br>
</div>
'''

map.get_root().html.add_child(folium.Element(legend_html))

# 지도 저장
map.save("result/fire_equip_map.html")

## 송파구 소방용수 시각화

In [8]:
fire_water

,순번,일련번호,수리번호,공설구분,용수형식,용수구분,수압,사용구분,관할서,안전센터,도로명,건물본번,건물부번,우편번호,좌표X,좌표Y,경위도X,경위도Y
0,1,110083413,440122,사설,지상식,소화전,NaN,양호,송파소방서,잠실119안전센터,올림픽로,300.0,0.0,NaN,209279.7409,445982.3528,127.104975,37.513220
1,2,110065754,140090,사설,지상식,소화전,3.0,양호,송파소방서,현장대응단,오금로,307.0,0.0,NaN,211193.2641,444880.8784,127.126605,37.503274
2,3,110066991,200450,공설,지상식,소화전,2.5,양호,송파소방서,거여119안전센터,오금로,524.0,0.0,NaN,212996.2247,443716.5910,127.146977,37.492759
3,4,110066992,200451,공설,지상식,소화전,3.0,양호,송파소방서,거여119안전센터,오금로,550.0,0.0,NaN,213257.1350,443702.1963,127.149928,37.492625
4,5,110066993,200461,공설,지상식,소화전,3.0,양호,송파소방서,거여119안전센터,오금로,540.0,0.0,NaN,213146.0496,443705.6892,127.148671,37.492658
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3812,3813,110084779,640205,사설,지상식,소화전,3.0,양호,송파소방서,가락119안전센터,법원로11길,25.0,0.0,NaN,199657.8231,450752.2932,126.996127,37.556249
3813,3814,110084905,140131,사설,지상식,소화전,NaN,양호,송파소방서,현장대응단,NaN,NaN,NaN,NaN,199657.8231,450752.2932,126.996127,37.556249
3814,3815,110061611,300269,공설,지하식,소화전,2.8,양호,송파소방서,방이119안전센터,바람드리9길,10.0,0.0,NaN,210532.6069,448773.1813,127.119188,37.538355
3815,3816,110058491,300555,공설,지하식,소화전,3.0,양호,송파소방서,방이119안전센터,올림픽로32길,36.0,18.0,NaN,209717.0360,445881.8552,127.109921,37.512310


In [9]:
fire_water['용수구분'].value_counts()

용수구분
소화전    3771
저수조      32
급수탑      14
Name: count, dtype: int64

In [10]:
grid = gpd.read_file("data/seoul_geo/seoul_geo.shp", encoding='utf-8')

In [11]:
grid

,id,pop,total_b,res_single,res_multi,b_over_20,EMD_CD,EMD_KOR_NM,geometry
0,1,364.0,13.0,3.0,4.0,NaN,11560111,당산동1가,"POLYGON ((126.89910 37.51987, 126.89909 37.520..."
1,2,448.0,21.0,3.0,18.0,16.0,11590108,대방동,"POLYGON ((126.92866 37.50109, 126.92866 37.501..."
2,3,426.0,2.0,NaN,2.0,NaN,11710109,장지동,"POLYGON ((127.13469 37.47576, 127.13469 37.476..."
3,4,215.0,10.0,1.0,6.0,NaN,11470103,신월동,"POLYGON ((126.84134 37.52497, 126.84133 37.525..."
4,5,52.0,17.0,3.0,NaN,13.0,11680105,삼성동,"POLYGON ((127.04400 37.51060, 127.04400 37.511..."
...,...,...,...,...,...,...,...,...,...
61642,61643,NaN,NaN,NaN,NaN,NaN,11350105,상계동,"POLYGON ((127.08835 37.67661, 127.08834 37.677..."
61643,61644,NaN,NaN,NaN,NaN,NaN,11500109,방화동,"POLYGON ((126.81818 37.58252, 126.81817 37.583..."
61644,61645,NaN,NaN,NaN,NaN,NaN,11350102,월계동,"POLYGON ((127.06374 37.61794, 127.06374 37.618..."
61645,61646,NaN,NaN,NaN,NaN,NaN,11650109,내곡동,"POLYGON ((127.06813 37.44669, 127.06813 37.447..."


In [12]:
songpa = grid[grid['EMD_CD'].astype(str).str.startswith('11710')]

In [13]:
songpa

,id,pop,total_b,res_single,res_multi,b_over_20,EMD_CD,EMD_KOR_NM,geometry
2,3,426.0,2.0,NaN,2.0,NaN,11710109,장지동,"POLYGON ((127.13469 37.47576, 127.13469 37.476..."
8,9,54.0,5.0,NaN,1.0,NaN,11710112,오금동,"POLYGON ((127.13227 37.50911, 127.13226 37.510..."
22,23,506.0,12.0,2.0,4.0,10.0,11710103,풍납동,"POLYGON ((127.11633 37.52708, 127.11633 37.527..."
38,39,617.0,43.0,13.0,30.0,25.0,11710106,삼전동,"POLYGON ((127.09158 37.49996, 127.09158 37.500..."
40,41,386.0,29.0,16.0,9.0,19.0,11710104,송파동,"POLYGON ((127.11304 37.50724, 127.11304 37.508..."
...,...,...,...,...,...,...,...,...,...
61555,61556,NaN,NaN,NaN,NaN,NaN,11710107,가락동,"POLYGON ((127.10747 37.49190, 127.10746 37.492..."
61571,61572,NaN,NaN,NaN,NaN,NaN,11710111,방이동,"POLYGON ((127.13905 37.51093, 127.13904 37.511..."
61579,61580,NaN,NaN,NaN,NaN,NaN,11710101,잠실동,"POLYGON ((127.07568 37.51162, 127.07567 37.512..."
61614,61615,NaN,NaN,NaN,NaN,NaN,11710108,문정동,"POLYGON ((127.11882 37.48292, 127.11882 37.483..."


In [14]:
import geopandas as gpd
import folium
import matplotlib.pyplot as plt
import matplotlib.colors  # matplotlib import 추가
import pandas as pd

In [15]:
from shapely.geometry import Point

# 경도와 위도를 이용해 Point 지오메트리 생성
fire_water['geometry'] = fire_water.apply(lambda row: Point(row['경위도X'], row['경위도Y']), axis=1)

# 새로운 geometry 열을 사용해 GeoDataFrame을 다시 생성
fire_water = gpd.GeoDataFrame(fire_water, geometry='geometry')

# CRS 설정
fire_water.set_crs(epsg=4326, inplace=True)

,순번,일련번호,수리번호,공설구분,용수형식,용수구분,수압,사용구분,관할서,안전센터,도로명,건물본번,건물부번,우편번호,좌표X,좌표Y,경위도X,경위도Y,geometry
0,1,110083413,440122,사설,지상식,소화전,NaN,양호,송파소방서,잠실119안전센터,올림픽로,300.0,0.0,NaN,209279.7409,445982.3528,127.104975,37.513220,POINT (127.10498 37.51322)
1,2,110065754,140090,사설,지상식,소화전,3.0,양호,송파소방서,현장대응단,오금로,307.0,0.0,NaN,211193.2641,444880.8784,127.126605,37.503274,POINT (127.12660 37.50327)
2,3,110066991,200450,공설,지상식,소화전,2.5,양호,송파소방서,거여119안전센터,오금로,524.0,0.0,NaN,212996.2247,443716.5910,127.146977,37.492759,POINT (127.14698 37.49276)
3,4,110066992,200451,공설,지상식,소화전,3.0,양호,송파소방서,거여119안전센터,오금로,550.0,0.0,NaN,213257.1350,443702.1963,127.149928,37.492625,POINT (127.14993 37.49263)
4,5,110066993,200461,공설,지상식,소화전,3.0,양호,송파소방서,거여119안전센터,오금로,540.0,0.0,NaN,213146.0496,443705.6892,127.148671,37.492658,POINT (127.14867 37.49266)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3812,3813,110084779,640205,사설,지상식,소화전,3.0,양호,송파소방서,가락119안전센터,법원로11길,25.0,0.0,NaN,199657.8231,450752.2932,126.996127,37.556249,POINT (126.99613 37.55625)
3813,3814,110084905,140131,사설,지상식,소화전,NaN,양호,송파소방서,현장대응단,NaN,NaN,NaN,NaN,199657.8231,450752.2932,126.996127,37.556249,POINT (126.99613 37.55625)
3814,3815,110061611,300269,공설,지하식,소화전,2.8,양호,송파소방서,방이119안전센터,바람드리9길,10.0,0.0,NaN,210532.6069,448773.1813,127.119188,37.538355,POINT (127.11919 37.53835)
3815,3816,110058491,300555,공설,지하식,소화전,3.0,양호,송파소방서,방이119안전센터,올림픽로32길,36.0,18.0,NaN,209717.0360,445881.8552,127.109921,37.512310,POINT (127.10992 37.51231)


In [16]:
# 공간 조인을 수행하여 각 songpa 그리드 내의 fire_water 수를 확인 (predicate 사용)
joined = gpd.sjoin(songpa, fire_water, how='left', predicate='contains')

In [17]:
joined

,id,pop,total_b,res_single,res_multi,b_over_20,EMD_CD,EMD_KOR_NM,geometry,index_right,...,관할서,안전센터,도로명,건물본번,건물부번,우편번호,좌표X,좌표Y,경위도X,경위도Y
2,3,426.0,2.0,NaN,2.0,NaN,11710109,장지동,"POLYGON ((127.13469 37.47576, 127.13469 37.476...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,9,54.0,5.0,NaN,1.0,NaN,11710112,오금동,"POLYGON ((127.13227 37.50911, 127.13226 37.510...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,23,506.0,12.0,2.0,4.0,10.0,11710103,풍납동,"POLYGON ((127.11633 37.52708, 127.11633 37.527...",2179.0,...,송파소방서,방이119안전센터,강동대로9길,16.0,0.0,NaN,210306.2195,447589.4167,127.116610,37.527690
22,23,506.0,12.0,2.0,4.0,10.0,11710103,풍납동,"POLYGON ((127.11633 37.52708, 127.11633 37.527...",2795.0,...,송파소방서,방이119안전센터,강동대로9길,20.0,0.0,NaN,210341.9076,447601.2877,127.117014,37.527797
38,39,617.0,43.0,13.0,30.0,25.0,11710106,삼전동,"POLYGON ((127.09158 37.49996, 127.09158 37.500...",1263.0,...,송파소방서,잠실119안전센터,백제고분로28길,27.0,19.0,NaN,208168.2363,444527.9047,127.092385,37.500125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61571,61572,NaN,NaN,NaN,NaN,NaN,11710111,방이동,"POLYGON ((127.13905 37.51093, 127.13904 37.511...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61579,61580,NaN,NaN,NaN,NaN,NaN,11710101,잠실동,"POLYGON ((127.07568 37.51162, 127.07567 37.512...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61614,61615,NaN,NaN,NaN,NaN,NaN,11710108,문정동,"POLYGON ((127.11882 37.48292, 127.11882 37.483...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61620,61621,NaN,8.0,NaN,NaN,NaN,11710101,잠실동,"POLYGON ((127.09945 37.50990, 127.09945 37.510...",987.0,...,송파소방서,잠실119안전센터,올림픽로,240.0,0.0,NaN,208862.4676,445673.7996,127.100251,37.510444


In [18]:
# 각 songpa 그리드별 fire_water 개수 집계
fire_water_count = joined['id'].value_counts().reset_index()
fire_water_count.columns = ['id', 'count']

In [19]:
fire_water_count

,id,count
0,7867,9
1,11667,8
2,8505,8
3,5562,6
4,12981,6
...,...,...
3453,33307,1
3454,33342,1
3455,33353,1
3456,33368,1


In [20]:
songpa = pd.merge(songpa, fire_water_count, on='id', how='left')
songpa

,id,pop,total_b,res_single,res_multi,b_over_20,EMD_CD,EMD_KOR_NM,geometry,count
0,3,426.0,2.0,NaN,2.0,NaN,11710109,장지동,"POLYGON ((127.13469 37.47576, 127.13469 37.476...",1
1,9,54.0,5.0,NaN,1.0,NaN,11710112,오금동,"POLYGON ((127.13227 37.50911, 127.13226 37.510...",1
2,23,506.0,12.0,2.0,4.0,10.0,11710103,풍납동,"POLYGON ((127.11633 37.52708, 127.11633 37.527...",2
3,39,617.0,43.0,13.0,30.0,25.0,11710106,삼전동,"POLYGON ((127.09158 37.49996, 127.09158 37.500...",1
4,41,386.0,29.0,16.0,9.0,19.0,11710104,송파동,"POLYGON ((127.11304 37.50724, 127.11304 37.508...",2
...,...,...,...,...,...,...,...,...,...,...
3453,61556,NaN,NaN,NaN,NaN,NaN,11710107,가락동,"POLYGON ((127.10747 37.49190, 127.10746 37.492...",2
3454,61572,NaN,NaN,NaN,NaN,NaN,11710111,방이동,"POLYGON ((127.13905 37.51093, 127.13904 37.511...",1
3455,61580,NaN,NaN,NaN,NaN,NaN,11710101,잠실동,"POLYGON ((127.07568 37.51162, 127.07567 37.512...",1
3456,61615,NaN,NaN,NaN,NaN,NaN,11710108,문정동,"POLYGON ((127.11882 37.48292, 127.11882 37.483...",1


In [21]:
songpa = songpa.rename(columns={'pop':'인구수',
                                'total_b':'건물수',
                               'res_single':'단독주택수',
                               'res_multi':'공동주택수',
                               'b_over_20':'노후건물수',
                               'EMD_CD':'읍면동코드',
                               'EMD_KOR_NM':'동',
                               'count':'소화용수',
                               'geometry':'geometry'})

In [22]:
songpa.head()

,id,인구수,건물수,단독주택수,공동주택수,노후건물수,읍면동코드,동,geometry,소화용수
0,3,426.0,2.0,NaN,2.0,NaN,11710109,장지동,"POLYGON ((127.13469 37.47576, 127.13469 37.476...",1
1,9,54.0,5.0,NaN,1.0,NaN,11710112,오금동,"POLYGON ((127.13227 37.50911, 127.13226 37.510...",1
2,23,506.0,12.0,2.0,4.0,10.0,11710103,풍납동,"POLYGON ((127.11633 37.52708, 127.11633 37.527...",2
3,39,617.0,43.0,13.0,30.0,25.0,11710106,삼전동,"POLYGON ((127.09158 37.49996, 127.09158 37.500...",1
4,41,386.0,29.0,16.0,9.0,19.0,11710104,송파동,"POLYGON ((127.11304 37.50724, 127.11304 37.508...",2


In [23]:
songpa['소화용수'].value_counts()

소화용수
1    2385
2     448
3     364
4     181
5      59
6      18
8       2
9       1
Name: count, dtype: int64

In [24]:
# 소화용수 레벨에 따른 색상 맵핑 함수
def get_color(firewater_level):
    if firewater_level == 1:
        return 'gray'
    elif firewater_level == 2:
        return '#030A8C'
    elif firewater_level <= 4:
        return '#4CD948'
    elif firewater_level <= 6:
        return '#F2DC6D'
    else:
        return '#F24B4B'

# 지도 객체 생성 (송파구 중심 좌표로 설정)
m = folium.Map(location=[37.514, 127.106], zoom_start=12, tiles='CartoDB positron')

for _, row in songpa.iterrows():
    folium.GeoJson(
        row['geometry'],
        style_function=lambda x, color=row['소화용수']: {'fillColor': get_color(color), 'color': 'black', 'weight': 0.1}  # 그리드 선을 더 얇게 처리
    ).add_to(m)

# 범례 추가
legend_html = '''
<div style="position: fixed; 
     bottom: 50px; left: 50px; width: 240px; height: 170px; 
     background-color: white; border:2px solid rgba(0,0,0,0.1); 
     z-index:9999; font-size:14px; border-radius: 10px; 
     box-shadow: 3px 3px 10px rgba(0,0,0,0.2); padding: 15px;">
     <h4 style="text-align: center; font-weight: bold; margin-top: 0;">소방용수 마커 색상</h4>
     <ul style="list-style: none; padding: 0.15; text-align: left;">
        <li><i style="background:gray; border-radius: 50%; width: 15px; height: 15px; display: inline-block; margin-right: 5px;"></i> 회색 - 1개</li>
        <li><i style="background:#030A8C; border-radius: 50%; width: 15px; height: 15px; display: inline-block; margin-right: 5px;"></i> 진한 파랑 - 2개</li>
        <li><i style="background:#4CD948; border-radius: 50%; width: 15px; height: 15px; display: inline-block; margin-right: 5px;"></i> 밝은 녹색 - 3~4개</li>
        <li><i style="background:#F2DC6D; border-radius: 50%; width: 15px; height: 15px; display: inline-block; margin-right: 5px;"></i> 노랑색 - 5~6개</li>
        <li><i style="background:#F24B4B; border-radius: 50%; width: 15px; height: 15px; display: inline-block; margin-right: 5px;"></i> 빨강 - 8~9개</li>
     </ul>
</div>
'''

m.get_root().html.add_child(folium.Element(legend_html))


# 지도 표시
m.save('result/songpa_firewater_map_with_legend.html')

In [29]:
songpa.to_file("data/songpa_grid.shp", encoding='euc-kr')
